In [22]:
%matplotlib inline
from __future__ import print_function

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
 
from utils.nolearn_net import NeuralNet
from utils.helper import *
#from utils.iterator import BaseBatchIterator
#from utils.utils import parse_conf_file
#from utils.preprocessing import build_pipeline
import importlib
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from utils.helper import *
from utils.train_utils import *

In [24]:
m = Model(config,mode ='train')

Loading the prepro pipeline : [u'MyImputer', u'MyStandardScaler']
Loading data iterator using : benchmark
Le train is composed by 21 group and 3738 observation
Le val is composed by 8 group and 1112 observation
Le test is composed by 5 group and 948 observation
Build the architecture: lstm, build_simple_lstm
Set up the checkpoints
Initialize the network 


In [25]:
m.nb_epochs =2

In [26]:
m.continue_training = False

In [30]:
m.train()

      7      10.32627       9.52514      1.08411  2.83s
      8       9.91381       9.30037      1.06596  2.76s
Loading best param
Loaded parameters to layer 'lstm' (shape 9x60).
Loaded parameters to layer 'lstm' (shape 60x60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 9x60).
Loaded parameters to layer 'lstm' (shape 60x60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 9x60).
Loaded parameters to layer 'lstm' (shape 60x60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 9x60).
Loaded parameters to layer 'lstm' (shape 60x60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 60).
Loaded parameters to layer 'lstm' (shape 1x60).
Loaded parameters to layer 'lstm' (shape 1x60).
Loaded parameters to layer 'dense' (shape 60x1).
Loaded paramet

In [39]:
m.predict_set('val')

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [73]:
for i,(Xb,yb) in enumerate(m.batch_ite_val):
    #rint(np.sqrt(np.sum((m.net.predict(Xb)-yb)*2)))
    print(m.net.get_score(Xb,yb))
    
    print(np.max(yb))
    #if i==0:
    #    sys.exit(1)
    

0.00189918775047
0.0
0.0086294509874
0.0
0.00506923594472
0.0
0.000143659867987
0.0
1.82097364442
9.47603121517
1.4666603469
9.47603121517
2.31026849406
10.1309921962
27.1281328162
19.5861204013
4.9626771481
19.5861204013
0.00238915669348
0.0
0.000624970059044
0.0
0.47153130055
0.0940635451509
0.00571924492744
0.0
2.70815562568
10.1031215162
0.00148028814621
0.0
0.928401325626
7.13489409141
61.9750997814
30.2745261984
103.023580933
30.2745261984
12.4923514271
13.6078595318
17.7244948423
13.6078595318
14.4262192887
15.2382943144
16.0870738566
20.8542363434
14.6375957747
20.8542363434
4.74302776666
14.6251393534
42.2724442336
21.4673913044
20.1062167735
21.4673913044
0.000419151974207
0.0
0.75813657003
0.515607580825
1.24488833456
0.515607580825
4.46475968686
8.64687848384
9.44132061525
8.64687848384
0.305763198472
0.00696767001117
0.0363338857375
0.0
1.46778860377
0.0
0.0155704500868
0.0
3.39175906716
2.4386845039
2.97323323476
2.4386845039
0.79764444381
5.70652173913
5.45489133406
5.70

In [66]:
np.mean(np.sum((m.net.predict(Xb)-yb)**2,axis=0))

20.064250186333723

In [71]:
mean_squared_error(m.net.predict(Xb).ravel(),yb.ravel())

1.1146805659074293

In [72]:
m.get_score_set('val')

3.0496205250770312

In [60]:
from sklearn.metrics import mean_squared_error

In [62]:
mean_squared_error(m.net.predict(Xb),yb)

1.1146805659074293

# Test 

In [11]:
config = parse_conf_file('/Users/cthorey/Documents/project/competition/fognet/models/ray/lstm/model_0/conf_model.json')

In [12]:
config

{u'architecture': u'build_simple_lstm',
 u'batch_size': 25,
 u'build_ite': u'benchmark',
 u'continue_training': False,
 u'feats': [u'percip_mm',
  u'humidity',
  u'temp',
  u'leafwet450_min',
  u'leafwet460_min',
  u'leafwet_lwscnt',
  u'gusts_ms',
  u'wind_dir',
  u'wind_ms'],
 u'grad_clip': 1,
 u'hiddens': 60,
 u'loss_function': u'squared_error',
 u'lr': 0.0001,
 u'model': u'lstm',
 u'name': u'micro',
 u'nb_epochs': 1000,
 u'overwrite': True,
 u'patience': 25,
 u'pipe_kwargs': {u'MyImputer__strategy': u'mean'},
 u'pipe_list': [u'MyImputer', u'MyStandardScaler'],
 u'platform': u'ray',
 u'reg': 1e-06,
 u'root': u'/Users/cthorey/Documents/project/competition/fognet/models/ray/lstm/model_0',
 u'seq_length': 25,
 u'stride': 1,
 u'update_rule': u'adam',
 u'verbose': 11}

In [ ]:
a = {'b':2,'a':1}
class D(object):
    def __init__(self,a):
        map(set)
        

In [381]:
pipeline = build_pipeline(config['pipe_list'], config['pipe_kwargs'])
nb_features, batch_ite_train, batch_ite_val, batch_ite_test, batch_ite_pred = load_data(
        name=config['name'],
        feats=config['feats'],
        build_ite=config['build_ite'],
        pipeline=pipeline)
model = importlib.import_module(
        'model_defs.%s' % config['model'])
builder = getattr(model, config['architecture'])
architecture = builder(D=nb_features, H=config[
        'hiddens'], grad_clip=config['grad_clip'])

Le train is composed by 21 group and 3738 observation
Le val is composed by 8 group and 1112 observation
Le test is composed by 5 group and 948 observation


In [385]:
net = NeuralNet(
        layers=architecture,
        regression=True,
        objective_loss_function=getattr(
            lasagne.objectives, config['loss_function']),
        objective_l2=config['reg'],  # L2 regularization
        update=getattr(lasagne.updates, config['update_rule']),
        update_learning_rate=config['lr'],
        batch_iterator_train=batch_ite_train,
        batch_iterator_test=batch_ite_val,
        verbose=config['verbose'],
        max_epochs=10000,)
net.initialize()
net.load_params_from(config['model_fname'])
with open(config['model_history_fname']) as f:
    net.train_history_ = pickle.load(f)

Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 9x25).
Loaded parameters to layer 'lstm' (shape 25x25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'lstm' (shape 1x25).
Loaded parameters to layer 'dense' (shape 25x1).
Loaded parameters to layer 'dense' (shape 1).


In [388]:
net.get_score_whole_set('train'),net.get_score_whole_set('test')

(3.0808924682517103, 5.8183123682635864)

In [389]:
from utils.hook import write_final_score

In [390]:
write_final_score(config,net)

In [395]:
from utils.prediction_utils import *

In [397]:
    final_pred = prediction(net, batch_ite_pred)
    make_submission(config, final_pred)

Will write output to /Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/submissions_20160322_144424.csv


In [440]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    for conf_run in conf_runs:
        config.update(conf_run)
        yield config

In [441]:
parameters_grid = {'lr':np.arange(1,10,1),
                  'reg':np.arange(1,10,1),
                  'hiddens':np.arange(1,10,1)}

In [435]:
    parameters_grid = {'lr': np.logspace(-7, -1, num=20),
                       'reg': np.logspace(-7, -1, num=20),
                       'hiddens': range(25, 525, 25)}
    confs = conf_generator(config, parameters_grid)

In [437]:
import multiprocessing

In [438]:
multiprocessing.cpu_count()

4

In [434]:
for i,elt in enumerate(confs):
    print(elt['lr'])
    if i>10:
        sys.exit()

2
2
2
2
2
2
3
3
3
3
3
3


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.


In [439]:

from multiprocessing import Pool 

In [449]:
parameters_grid

{'hiddens': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'lr': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'reg': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}

In [453]:
def test(config):
    print(config['lr'],config['reg'],config['hiddens'])

In [ ]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    for conf_run in conf_runs:
        config.update(conf_run)
        yield config

In [459]:
for elt in confs:
    print(elt)

9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 

In [457]:
confs = conf_generator(config,parameters_grid)

In [458]:
pool = Pool()
pool.map(test, confs)
pool.close() 
pool.join()

9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 9
9 9 

In [476]:
    parameters_grid = {'lr': np.logspace(-3, -2, num=2),
                       'reg': [1e-7],
                       'hiddens': [25, 1,50]}

In [523]:
class MyDict(dict):

    def update(self, *args):
        dict.update(self, *args)
        return self

In [546]:
def conf_generator(config, parameters_grids):

    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    confs = map(lambda d: update_dict(config, d), conf_runs)
    return confs


In [548]:
confs = conf_generator(config,parameters_grid)

In [549]:
for elt in confs:
    print(elt['lr'],elt['hiddens'],elt['reg'])

0.001 25 1e-07
0.01 25 1e-07
0.001 1 1e-07
0.01 1 1e-07
0.001 50 1e-07
0.01 50 1e-07


In [515]:
def worker(elt):
    print(elt['lr'],elt['hiddens'],elt['reg'])

In [550]:
confs = conf_generator(config,parameters_grid)
pool = Pool()
pool.map(worker, ite)
pool.close()
pool.join()

0.001 25 1e-07
0.01 25 1e-07
0.001 1 1e-07
0.01 1 1e-07
0.001 50 1e-07
0.01 50 1e-07


In [494]:
class myDict(dict):
    def update(self, *args):
    	dict.update(self, *args)
    	return self

In [496]:
myconfig = myDict(config)

In [498]:
map(myconfig.update,ite)

[{u'architecture': u'build_simple_lstm',
  u'build_ite': u'benchmark',
  u'conf': u'../models/lstm/model_3/conf_model.json',
  u'continue_training': False,
  u'feats': [u'percip_mm',
   u'humidity',
   u'temp',
   u'leafwet450_min',
   u'leafwet460_min',
   u'leafwet_lwscnt',
   u'gusts_ms',
   u'wind_dir',
   u'wind_ms'],
  u'folder': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001',
  u'grad_clip': 50,
  u'hiddens': 50,
  u'loss_function': u'squared_error',
  u'lr': 0.01,
  u'model': u'lstm',
  u'model_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model.pkl',
  u'model_graph_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.png',
  u'model_history_fname': u'/Users/thorey/Documents/project/competition/fognet/models/lstm/model_3/h_25_rg_1e-07_lr_0.001/model_history.pkl',
  u'model_name': u'h_25_rg_1e-07_lr_0.001',


In [492]:
def update_dict(config,d):
    config.update(d)
    return config


In [493]:
map(update_dict,ite)

TypeError: update_dict() takes exactly 2 arguments (1 given)

In [501]:
class myDict(dict):

    def update(self, *args):
        dict.update(self, *args)
        return self


def conf_generator(config, parameters_grids):

    config = myDict(config)
    assert all([f in ['lr', 'reg', 'hiddens'] for f in parameters_grid.keys()])
    product = [x for x in apply(itertools.product, parameters_grid.values())]
    conf_runs = [dict(zip(parameters_grid.keys(), p)) for p in product]
    conf_runs = map(dict, map(config.update, conf_runs))
    return conf_runs

In [503]:
confs = conf_generator(config,parameters_grid)

In [504]:
len(confs)

6

In [505]:
pool = Pool()
pool.map(worker, confs)
pool.close()
pool.join()

0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07
0.01 50 1e-07


In [552]:
multiprocessing.cpu_count()

4